In [27]:
from __future__ import print_function
import argparse
import sys

import bosdyn.client
import bosdyn.client.estop
import bosdyn.client.lease
import bosdyn.client.util

from bosdyn.client.robot_command import RobotCommandClient, blocking_stand
from bosdyn.client.manipulation_api_client import ManipulationApiClient
from bosdyn.api import manipulation_api_pb2
from bosdyn.client.frame_helpers import *
from bosdyn.api import geometry_pb2
from bosdyn.client.robot_state import RobotStateClient
from bosdyn.client.image import ImageClient
from bosdyn.api import image_pb2
from bosdyn.client.estop import EstopClient
from bosdyn.api import estop_pb2

import traceback
import time

import numpy as np
import cv2



from __future__ import print_function
import argparse
import sys
import time
import os
import bosdyn.client
import bosdyn.client.lease
import bosdyn.client.util
import bosdyn.geometry

from bosdyn.client.image import ImageClient
from bosdyn.client.robot_command import RobotCommandBuilder, RobotCommandClient, blocking_stand


In [29]:
sdk = bosdyn.client.create_standard_sdk('HelloSpotClient')
robot = sdk.create_robot("192.168.80.3")
robot.authenticate("user", "spot_locomotion")
robot_state_client = robot.ensure_client(RobotStateClient.default_service_name)
robot_state = robot_state_client.get_robot_state

# Establish time sync with the robot. This kicks off a background thread to establish time sync.
# Time sync is required to issue commands to the robot. After starting time sync thread, block
# until sync is established.
robot.time_sync.wait_for_sync()



In [5]:
import pickle

In [40]:
#traj = []

for i in range(1000):
    robot_state = robot_state_client.get_robot_state()
    traj.append(robot_state)

with open("recording_1.pkl", "wb") as f:
    pickle.dump(traj, f)

In [8]:
print(traj[100])

power_state {
  motor_power_state: STATE_ON
  shore_power_state: STATE_OFF_SHORE_POWER
  locomotion_charge_percentage {
    value: 6.0
  }
  locomotion_estimated_runtime {
    seconds: 377
    nanos: 773925781
  }
  robot_power_state: ROBOT_POWER_STATE_ON
  payload_ports_power_state: PAYLOAD_PORTS_POWER_STATE_ON
  wifi_radio_power_state: WIFI_RADIO_POWER_STATE_ON
}
battery_states {
  identifier: "02-21608-002e"
  charge_percentage {
    value: 6.0
  }
  estimated_runtime {
    seconds: 377
    nanos: 773925781
  }
  current {
    value: -6.4800004959106445
  }
  voltage {
    value: 44.94000244140625
  }
  temperatures: 40.20695877075195
  temperatures: 40.0616455078125
  temperatures: 40.871864318847656
  temperatures: 40.63408279418945
  temperatures: 39.703704833984375
  temperatures: 40.75737380981445
  temperatures: 40.995155334472656
  temperatures: 40.75297164916992
  temperatures: 40.80141067504883
  temperatures: 40.26420211791992
  status: STATUS_DISCHARGING
}
comms_states {


In [43]:
traj_rot =[f.kinematic_state.transforms_snapshot.child_to_parent_edge_map["flat_body"].parent_tform_child.rotation for f in traj]

In [36]:
class X:
    
    def __init__(self, q):
        self._q = q
    def to_quaternion(self):
        return self._q

In [46]:
lease_client = robot.ensure_client(bosdyn.client.lease.LeaseClient.default_service_name)
# lease = lease_client.acquire()
lease = lease_client.take()

with bosdyn.client.lease.LeaseKeepAlive(lease_client):
    robot.logger.info("Powering on robot... This may take several seconds.")
    robot.power_on(timeout_sec=20)
    assert robot.is_powered_on(), "Robot power on failed."
    robot.logger.info("Robot powered on.")
    
    # Tell the robot to stand up. The command service is used to issue commands to a robot.
    # The set of valid commands for a robot depends on hardware configuration. See
    # SpotCommandHelper for more detailed examples on command building. The robot
    # command service requires timesync between the robot and the client.
    robot.logger.info("Commanding robot to stand...")
    command_client = robot.ensure_client(RobotCommandClient.default_service_name)
    blocking_stand(command_client, timeout_sec=10)
    robot.logger.info("Robot standing.")
    time.sleep(1)
    
    for i in range(500):
    #   print(i)
    #   yaw = 0.4 * np.sin(0.1 * i)
    #   pitch = 0.2  * np.cos(0.2 * i)
    #   roll = 0.1 * np.sin(0.3 * i)
      # footprint_R_body = bosdyn.geometry.EulerZXY(yaw=yaw, roll=roll, pitch=pitch)
      cmd = RobotCommandBuilder.synchro_stand_command(footprint_R_body=X(traj_rot[i]))
      command_client.robot_command(cmd)
      robot.logger.info("Robot standing twisted.")
      #time.sleep(0.01)

Generic exception for  during check-in:
bosdyn.api.RetainLeaseResponse (LeaseUseError): 
    (resuming check-in)
Generic exception for  during check-in:
bosdyn.api.RetainLeaseResponse (LeaseUseError): 
    (resuming check-in)


In [20]:
x=bosdyn.geometry.Quaternion()

In [21]:
type(traj[100].kinematic_state.transforms_snapshot.child_to_parent_edge_map["flat_body"].parent_tform_child.rotation)

bosdyn.api.geometry_pb2.Quaternion

In [23]:
type(bosdyn.geometry.EulerZXY())

bosdyn.geometry.EulerZXY

In [24]:
RobotCommandBuilder.synchro_stand_command

NameError: name 'RobotCommandBuilder' is not defined

In [52]:
lease_client = robot.ensure_client(bosdyn.client.lease.LeaseClient.default_service_name)
# lease = lease_client.acquire()
lease = lease_client.take()

with bosdyn.client.lease.LeaseKeepAlive(lease_client):
    robot.logger.info("Powering on robot... This may take several seconds.")
    robot.power_on(timeout_sec=20)
    assert robot.is_powered_on(), "Robot power on failed."
    robot.logger.info("Robot powered on.")
    
    # Tell the robot to stand up. The command service is used to issue commands to a robot.
    # The set of valid commands for a robot depends on hardware configuration. See
    # SpotCommandHelper for more detailed examples on command building. The robot
    # command service requires timesync between the robot and the client.
    robot.logger.info("Commanding robot to stand...")
    command_client = robot.ensure_client(RobotCommandClient.default_service_name)
    # blocking_stand(command_client, timeout_sec=10)
    robot.logger.info("Robot standing.")
    time.sleep(1)

    # batt_change_command = RobotCommandBuilder.battery_change_pose_command()
    batt_change_command = RobotCommandBuilder.selfright_command()
    command_client.robot_command(batt_change_command)
    
    time.sleep(10)

Generic exception for  during check-in:
bosdyn.api.RetainLeaseResponse (LeaseUseError): 
    (resuming check-in)
Error refreshing the token.  Retry in 0:00:01
Traceback (most recent call last):
  File "/home/pengxu/Work/spot/my_spot_env/lib/python3.9/site-packages/bosdyn/client/token_manager.py", line 62, in update
    self.robot.authenticate_with_token(self.robot.user_token)
  File "/home/pengxu/Work/spot/my_spot_env/lib/python3.9/site-packages/bosdyn/client/robot.py", line 357, in authenticate_with_token
    user_token = auth_client.auth_with_token(token, self.app_token, timeout=timeout)
  File "/home/pengxu/Work/spot/my_spot_env/lib/python3.9/site-packages/bosdyn/client/auth.py", line 145, in auth_with_token
    return self.call(self._stub.GetAuthToken, req, _token_from_response, _error_from_response,
  File "/home/pengxu/Work/spot/my_spot_env/lib/python3.9/site-packages/bosdyn/client/common.py", line 234, in processor
    return func(self, rpc_method, request, value_from_response=v

Error refreshing the token.  Retry in 0:02:08
Traceback (most recent call last):
  File "/home/pengxu/Work/spot/my_spot_env/lib/python3.9/site-packages/bosdyn/client/token_manager.py", line 62, in update
    self.robot.authenticate_with_token(self.robot.user_token)
  File "/home/pengxu/Work/spot/my_spot_env/lib/python3.9/site-packages/bosdyn/client/robot.py", line 357, in authenticate_with_token
    user_token = auth_client.auth_with_token(token, self.app_token, timeout=timeout)
  File "/home/pengxu/Work/spot/my_spot_env/lib/python3.9/site-packages/bosdyn/client/auth.py", line 145, in auth_with_token
    return self.call(self._stub.GetAuthToken, req, _token_from_response, _error_from_response,
  File "/home/pengxu/Work/spot/my_spot_env/lib/python3.9/site-packages/bosdyn/client/common.py", line 234, in processor
    return func(self, rpc_method, request, value_from_response=value_from_response,
  File "/home/pengxu/Work/spot/my_spot_env/lib/python3.9/site-packages/bosdyn/client/common.p

Error refreshing the token.  Retry in 0:00:08
Traceback (most recent call last):
  File "/home/pengxu/Work/spot/my_spot_env/lib/python3.9/site-packages/bosdyn/client/token_manager.py", line 62, in update
    self.robot.authenticate_with_token(self.robot.user_token)
  File "/home/pengxu/Work/spot/my_spot_env/lib/python3.9/site-packages/bosdyn/client/robot.py", line 357, in authenticate_with_token
    user_token = auth_client.auth_with_token(token, self.app_token, timeout=timeout)
  File "/home/pengxu/Work/spot/my_spot_env/lib/python3.9/site-packages/bosdyn/client/auth.py", line 145, in auth_with_token
    return self.call(self._stub.GetAuthToken, req, _token_from_response, _error_from_response,
  File "/home/pengxu/Work/spot/my_spot_env/lib/python3.9/site-packages/bosdyn/client/common.py", line 234, in processor
    return func(self, rpc_method, request, value_from_response=value_from_response,
  File "/home/pengxu/Work/spot/my_spot_env/lib/python3.9/site-packages/bosdyn/client/common.p

Error refreshing the token.  Retry in 0:00:04
Traceback (most recent call last):
  File "/home/pengxu/Work/spot/my_spot_env/lib/python3.9/site-packages/bosdyn/client/token_manager.py", line 62, in update
    self.robot.authenticate_with_token(self.robot.user_token)
  File "/home/pengxu/Work/spot/my_spot_env/lib/python3.9/site-packages/bosdyn/client/robot.py", line 357, in authenticate_with_token
    user_token = auth_client.auth_with_token(token, self.app_token, timeout=timeout)
  File "/home/pengxu/Work/spot/my_spot_env/lib/python3.9/site-packages/bosdyn/client/auth.py", line 145, in auth_with_token
    return self.call(self._stub.GetAuthToken, req, _token_from_response, _error_from_response,
  File "/home/pengxu/Work/spot/my_spot_env/lib/python3.9/site-packages/bosdyn/client/common.py", line 234, in processor
    return func(self, rpc_method, request, value_from_response=value_from_response,
  File "/home/pengxu/Work/spot/my_spot_env/lib/python3.9/site-packages/bosdyn/client/common.p

Error refreshing the token.  Retry in 0:04:16
Traceback (most recent call last):
  File "/home/pengxu/Work/spot/my_spot_env/lib/python3.9/site-packages/bosdyn/client/token_manager.py", line 62, in update
    self.robot.authenticate_with_token(self.robot.user_token)
  File "/home/pengxu/Work/spot/my_spot_env/lib/python3.9/site-packages/bosdyn/client/robot.py", line 357, in authenticate_with_token
    user_token = auth_client.auth_with_token(token, self.app_token, timeout=timeout)
  File "/home/pengxu/Work/spot/my_spot_env/lib/python3.9/site-packages/bosdyn/client/auth.py", line 145, in auth_with_token
    return self.call(self._stub.GetAuthToken, req, _token_from_response, _error_from_response,
  File "/home/pengxu/Work/spot/my_spot_env/lib/python3.9/site-packages/bosdyn/client/common.py", line 234, in processor
    return func(self, rpc_method, request, value_from_response=value_from_response,
  File "/home/pengxu/Work/spot/my_spot_env/lib/python3.9/site-packages/bosdyn/client/common.p